**Unet architecture**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)

    s2, p2 = encoder_block(p1, 128)

    s3, p3 = encoder_block(p2, 256)

    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)

    d2 = decoder_block(d1, s3, 256)

    d3 = decoder_block(d2, s2, 128)

    d4 = decoder_block(d3, s1, 64)


    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 1)
    model = build_unet(input_shape,9)
    model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         640       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0][0

**Training**

In [ ]:
from glob import glob
import os
import numpy as np
import cv2
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from google.colab.patches import cv2_imshow
import scipy.io
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.optimizers import Adam as LegacyAdam
""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def load_dataset(path, split=0.2):
  data = sorted(glob(os.path.join(path,"*")))
  print(len(data))
  split_size=int(split*len(data))
  train, valid = train_test_split(data, test_size=split_size, random_state=42)
  train, test = train_test_split(data, test_size=split_size, random_state=42)

  return train, valid, test


def read_image_label(x):
  x = np.load(x)
  img = x['image']
  label = x['label']
  img = np.expand_dims(img,axis = -1);
  label = np.expand_dims(label,axis = -1);
  assert img.shape == label.shape
  """ Image processing """
  img = img.astype(np.float32)
  """ label processing """
  output = []
  for color in COLORMAP:
    cmap = np.all(np.equal(label, color), axis=-1)
    output.append(cmap)
  output = np.stack(output, axis=-1)
  output = output.astype(np.uint8)

  return img,output


def preprocess(x):
    def f(x):
        x = x.decode()
        image, mask = read_image_label(x)
        return image, mask

    image, mask = tf.numpy_function(f,[x], [tf.float32, tf.uint8])
    image.set_shape([512, 512, 1])
    mask.set_shape([512, 512, 9])

    return image, mask

def tf_dataset(x, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices(x)
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("results")

    """ Hyperparameters """
    IMG_H = 512
    IMG_W = 512
    NUM_CLASSES = 9
    input_shape = (IMG_H, IMG_W, 1)

    batch_size = 6
    lr = 1e-4
    num_epochs = 20

    model_path = os.path.join("results", "model.h5")
    csv_path = os.path.join("results", "data.csv")

    path = "/content/drive/MyDrive/train_npz"

    train , valid , test = load_dataset(path)

    mat_file = scipy.io.loadmat('MOS.mat')

    COLORMAP = mat_file['colormap']

    print(f"train:{len(train)} , test: {len(test)}, valid :{len(valid)}")

    train_dataset = tf_dataset(train, batch=8)
    valid_dataset = tf_dataset(valid, batch=8)

    print(train_dataset);
    """ Model """

    model = build_unet(input_shape, NUM_CLASSES)
    # model.load_weights(model_path)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.legacy.Adam(lr)
    )
    model.summary()

    """ Training """
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=num_epochs,
        callbacks=callbacks
    )



2210
train:1768 , test: 442, valid :442
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 512, 512, 9), dtype=tf.uint8, name=None))>
Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 512, 512, 64)         640       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, 512, 512, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                   

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


221/221 [==============================] - 537s 2s/step - loss: 0.9253 - val_loss: 0.9028 - lr: 1.0000e-04
Epoch 2/20
221/221 [==============================] - ETA: 0s - loss: 0.4103
Epoch 2: val_loss improved from 0.90284 to 0.40393, saving model to results/model.h5
221/221 [==============================] - 513s 2s/step - loss: 0.4103 - val_loss: 0.4039 - lr: 1.0000e-04
Epoch 3/20
221/221 [==============================] - ETA: 0s - loss: 0.2637
Epoch 3: val_loss improved from 0.40393 to 0.25059, saving model to results/model.h5
221/221 [==============================] - 490s 2s/step - loss: 0.2637 - val_loss: 0.2506 - lr: 1.0000e-04
Epoch 4/20
221/221 [==============================] - ETA: 0s - loss: 0.1841
Epoch 4: val_loss improved from 0.25059 to 0.15305, saving model to results/model.h5
221/221 [==============================] - 491s 2s/step - loss: 0.1841 - val_loss: 0.1530 - lr: 1.0000e-04
Epoch 5/20
221/221 [==============================] - ETA: 0s - loss: 0.1340
Epoch 5: 

**Testing**

In [ ]:
from __future__ import print_function
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
import matplotlib.pyplot as plt
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred)
    return 2.0 * intersection / union

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.2):
  data = sorted(glob(os.path.join(path,"*")))
  print(len(data))
  split_size=int(split*len(data))
  train, valid = train_test_split(data, test_size=split_size, random_state=42)
  train, test = train_test_split(data, test_size=split_size, random_state=42)

  return train, valid, test

def predict(data):
  x = np.load(data)
  image = x['image']
  label = x['label']

  # prediction
  img = np.expand_dims(image, axis=0)
  """ Model """
  model_path = "/content/drive/MyDrive/MOS_model_50.h5"
  model = tf.keras.models.load_model(model_path)
  """ Prediction """
  pred = model.predict(img, verbose=0)[0]
  pred = np.argmax(pred, axis=-1)
  pred = pred.astype(np.float32)

  cmap = {
    0.0 : [0,0,0],
    0.1 : [255,0,0],
    0.2 : [255,255,1],
    0.3 : [0,0,255],
    0.4 : [0,255,1],
    0.5 : [255,0,255],
    0.6 : [0,255,255],
    0.7 : [244,208,63],
    0.8 : [234,240,241]
  }

  colored_label = np.array([[cmap[pixel/10] for pixel in row] for row in label])
  colored_Prediction = np.array([[cmap[pixel/10] for pixel in row] for row in pred ])

  #  rgb , rotate  & flip
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
  image = cv2.flip(image, 1)
  colored_label = cv2.rotate(colored_label, cv2.ROTATE_90_CLOCKWISE)
  colored_label = cv2.flip(colored_label, 1)
  colored_Prediction = cv2.rotate(colored_Prediction, cv2.ROTATE_90_CLOCKWISE)
  colored_Prediction = cv2.flip(colored_Prediction, 1)

  image = image*255
  alpha = 0.5
  image1 = alpha * image + (1 - alpha) * colored_label
  image2 = alpha * image + (1 - alpha) * colored_Prediction
  # cv2_imshow(image)
  # cv2_imshow(image1)
  # cv2_imshow(image2)

  return image, image1, image2
  # print(f"Image = {image.shape}")
  # print(f"Image1 = {image1.shape}")
  # print(f"Image2 = {image2.shape}")

create_dir("pred_imgs_view")
path = "/content/drive/MyDrive/train_npz"
train , valid , test = load_dataset(path)
print(f"train:{len(train)} , test: {len(test)}, valid :{len(valid)}")

# reading data of size (512 X 512) grayscale both image and label
data_test = test
SCORE = []
f = 0;
for data in test[:200] :
  img,g_truth,prediction = predict(data);
  res = np.concatenate([img,g_truth,prediction])
  cv2.imwrite(f"/content/pred_imgs_view/{f}.png",res)
  f = f+1

  g_truth    = g_truth.flatten().astype(np.int32)
  prediction = prediction.flatten().astype(np.int32)

  labels = [i for i in range(9)]

  f1_value = f1_score(g_truth, prediction, labels=labels, average=None, zero_division=0)
  jac_value = jaccard_score(g_truth, prediction, labels=labels, average=None, zero_division=0)
  SCORE.append([f1_value, jac_value])


score = np.array(SCORE)
score = np.mean(score, axis=0)
print(score.shape)

classes= {
    0:"Background",
    1:"aorta",
    2:"right kidney",
    3:"left kidney",
    4:"gallbladder",
    5:"liver",
    6:"pancreas",
    7:"spleen",
    8:"stomach"
 }
for j in range(9):
  print(f"{classes[j]:15s} = F1:{score[0][j]:10s} , jac :{score[1][j]:10s}")



2210
train:1768 , test: 442, valid :442


(2, 9)


ValueError: ignored

In [ ]:
print(score.shape)
classes= {
    0:"Background",
    1:"aorta",
    2:"right kidney",
    3:"left kidney",
    4:"gallbladder",
    5:"liver",
    6:"pancreas",
    7:"spleen",
    8:"stomach"
 }
for j in range(9):
  print(f"{classes[j]:15s} = DCS: {score[0][j]} , jac : {score[1][j]}")

(2, 9)
Background      = DCS: 0.9999906848112584 , jac : 0.9999813721141281
aorta           = DCS: 0.9997785110396673 , jac : 0.9995586073890874
right kidney    = DCS: 0.9997983408369022 , jac : 0.999599025042774
left kidney     = DCS: 0.9998478711322994 , jac : 0.999696504122554
gallbladder     = DCS: 0.9998174552032542 , jac : 0.9996361839732293
liver           = DCS: 0.9998324002512908 , jac : 0.9996654838556134
pancreas        = DCS: 0.9998563455011684 , jac : 0.9997131509614731
spleen          = DCS: 0.9998353360164097 , jac : 0.9996712979907109
stomach         = DCS: 0.9998117677356336 , jac : 0.9996242526717996
